# 🤖 Persistent Memory Chatbot with DynamoDB Saver

## 🎯 **Demo Overview**

This notebook demonstrates how to build an **intelligent chatbot with persistent memory** using:

- **🧠 LangGraph** for conversation workflow management
- **🗄️ DynamoDBSaver** for persistent state storage
- **🤖 Amazon Bedrock Claude** for natural language processing
- **🔄 Advanced Context Framing** to maintain conversation continuity

### ✨ **Key Features Demonstrated:**

1. **Persistent Memory Across Sessions**: Conversations survive application restarts
2. **Intelligent Summarization**: Long conversations are automatically summarized
3. **Cross-Instance Memory**: New graph instances access previous conversations
4. **Production-Ready Architecture**: Scalable, reliable memory management with AWS DynamoDB
5. **S3 Offloading**: Automatic offloading of large checkpoints (>350KB) to S3

### 🚀 **What Makes This Work:**

- **Complete Conversation History**: LLM receives full context in each request
- **Smart Context Framing**: Presents history as "ongoing conversation" not "memory"
- **DynamoDB Persistence**: Reliable, scalable state storage and retrieval
- **Automatic State Management**: Seamless message accumulation and retrieval

## 📋 Prerequisites & Setup

In [140]:
# Install required packages
# Base package with Dynamodb support:
# !pip install 'langgraph-checkpoint-aws'
#
# Individual packages, for langgraph application:
# !pip install langchain-aws langgraph langchain

import os
import getpass
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, RemoveMessage
from langchain_aws import ChatBedrockConverse
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Import DynamoDB saver
from langgraph_checkpoint_aws import DynamoDBSaver
import boto3

print("✅ All dependencies imported successfully!")
print("🗄️ DynamoDB saver ready for persistent memory")

✅ All dependencies imported successfully!
🗄️ DynamoDB saver ready for persistent memory


In [161]:
# Set AWS region
aws_region = input("AWS Region name (default: us-east-1): ") or "us-east-1"
os.environ["AWS_DEFAULT_REGION"] = aws_region

# Use existing AWS profile
aws_profile = input("AWS Profile name (default: default): ") or "default"
os.environ['AWS_PROFILE'] = aws_profile

boto_session = boto3.Session(
    profile_name=os.environ['AWS_PROFILE'],
    region_name=os.environ["AWS_DEFAULT_REGION"]
)

print(f"\n✅ Using AWS profile: {aws_profile} in region: {os.environ.get('AWS_DEFAULT_REGION')}")


✅ Using AWS profile: default in region: us-east-1


## 🗄️ DynamoDB Setup

**Prerequisites:**

1. **Deploy CloudFormation Stack**: Use the provided template to create DynamoDB table and S3 bucket
2. **AWS Credentials**: Ensure you have AWS credentials configured
3. **IAM Permissions**: Required permissions for DynamoDB and S3 (if using offloading)

In [163]:
import json
import uuid

print("🚀 DynamoDB Setup Instructions:")
# Generate random names
default_stack = f"langgraph-checkpoint-stack-{uuid.uuid4().hex[:8]}"
default_table = f"langgraph-checkpoints-ddb-{uuid.uuid4().hex[:8]}"
default_bucket = f"langgraph-checkpoints-s3-{uuid.uuid4().hex[:8]}"

# Get user input or use defaults
stack_name = input(f"Stack name (default: {default_stack}): ") or default_stack
table_name = input(f"Table name (default: {default_table}): ") or default_table
bucket_name = input(f"Bucket name (default: {default_bucket}): ") or default_bucket

# Read CloudFormation template
template_path = f"{os.getcwd()}/cfn/langgraph-ddb-cfn-template.yaml"
with open(template_path, 'r') as f:
    template_body = f.read()

# Deploy CloudFormation Stack
print("\n1. Deploy CloudFormation Stack:")
cfn = boto_session.client('cloudformation', region_name=aws_region)

try:
    response = cfn.create_stack(
        StackName=stack_name,
        TemplateBody=template_body,
        Parameters=[
            {'ParameterKey': 'CheckpointTableName', 'ParameterValue': table_name},
            {'ParameterKey': 'EnableTTL', 'ParameterValue': 'true'},
            {'ParameterKey': 'S3BucketName', 'ParameterValue': bucket_name},
            {'ParameterKey': 'CreateS3Bucket', 'ParameterValue': 'true'}
        ]
    )
    print(f"✅ Stack creation initiated: {response['StackId']}")
except ClientError as e:
    print(f"❌ Error: {e.response['Error']['Message']}")
    raise

# Wait for stack creation
print(f"\n2. Waiting for stack '{stack_name}' creation...")
waiter = cfn.get_waiter('stack_create_complete')
try:
    waiter.wait(StackName=stack_name, WaiterConfig={'Delay': 10, 'MaxAttempts': 60})
except Exception as e:
    print(f"❌ Stack creation failed: {e}")
    raise


# Get stack outputs and parse them
# Get stack outputs
print("\n3. Retrieving stack outputs...")
stack_info = cfn.describe_stacks(StackName=stack_name)
outputs = stack_info['Stacks'][0].get('Outputs', [])

TABLE_NAME = next((o['OutputValue'] for o in outputs if o['OutputKey'] == 'CheckpointTableName'), None)
S3_BUCKET_NAME = next((o['OutputValue'] for o in outputs if o['OutputKey'] == 'S3BucketName'), None)

if not TABLE_NAME:
    raise ValueError("❌ Failed to retrieve DynamoDB table name from stack outputs")

print("\n✅ CloudFormation stack created successfully!")
print(f"📊 DynamoDB Table: {TABLE_NAME}")
print(f"🪣 S3 Bucket: {S3_BUCKET_NAME}")

os.environ['DYNAMODB_TABLE_NAME'] = TABLE_NAME
os.environ['S3_BUCKET_NAME'] = S3_BUCKET_NAME or ""

🚀 DynamoDB Setup Instructions:

1. Deploy CloudFormation Stack:
✅ Stack creation initiated: arn:aws:cloudformation:<region>:<account_id>:stack/langgraph-checkpoint-stack-34693a00/72ace050-b670-11f0-b296-126ef5738c4b

2. Waiting for stack 'langgraph-checkpoint-stack-34693a00' creation...

3. Retrieving stack outputs...

✅ CloudFormation stack created successfully!
📊 DynamoDB Table: langgraph-checkpoints-ddb-717c7213
🪣 S3 Bucket: langgraph-checkpoints-s3-717d768c


## 🏗️ Architecture Setup

In [164]:
# Define conversation state with automatic message accumulation
class State(TypedDict):
    """Conversation state with persistent memory."""
    messages: Annotated[Sequence[BaseMessage], add_messages]  # Auto-accumulates messages
    summary: str  # Conversation summary for long histories

print("✅ State schema defined with automatic message accumulation")

✅ State schema defined with automatic message accumulation


In [165]:
# DynamoDBSaver configuration
REGION_NAME = os.environ["AWS_DEFAULT_REGION"]
TABLE_NAME = os.environ['DYNAMODB_TABLE_NAME'] 
S3_BUCKET_NAME = os.environ.get("S3_BUCKET_NAME", None)
TTL_SECONDS = 3600  # 1 hour TTL for demo

# Initialize language model
model = ChatBedrockConverse(
    model="global.anthropic.claude-sonnet-4-20250514-v1:0",
    temperature=0.7,
    max_tokens=2048,
    region_name=REGION_NAME,
    client=boto_session.client('bedrock-runtime')
)

print("✅ Language model initialized (Claude 4 Sonnet)")
print(f"✅ DynamoDB configured in {REGION_NAME}: {TABLE_NAME} with {TTL_SECONDS/3600}h TTL")
if S3_BUCKET_NAME:
    print(f"✅ S3 offloading enabled: {S3_BUCKET_NAME}")

✅ Language model initialized (Claude 4 Sonnet)
✅ DynamoDB configured in us-east-1: langgraph-checkpoints-ddb-717c7213 with 1.0h TTL
✅ S3 offloading enabled: langgraph-checkpoints-s3-717d768c


## 🧠 Enhanced Memory Logic

The key to persistent memory is **intelligent context framing** that avoids triggering Claude's memory denial training.

In [166]:
def call_model_with_memory(state: State):
    """Enhanced LLM call with intelligent context framing for persistent memory."""
    
    # Get conversation components
    summary = state.get("summary", "")
    messages = state["messages"]
    
    print(f"🧠 Processing {len(messages)} messages | Summary: {'✅' if summary else '❌'}")
    
    # ENHANCED: Intelligent context framing
    if summary and len(messages) > 2:
        # Create natural conversation context using summary
        system_message = SystemMessage(
            content=f"You are an AI assistant in an ongoing conversation. "
                   f"Here's what we've discussed so far: {summary}\n\n"
                   f"Continue the conversation naturally, building on what was previously discussed. "
                   f"Don't mention memory or remembering - just respond as if this is a natural conversation flow."
        )
        # Use recent messages with enhanced context
        recent_messages = list(messages[-4:])  # Last 4 messages for immediate context
        full_messages = [system_message] + recent_messages
    elif len(messages) > 6:
        # For long conversations without summary, use recent messages
        system_message = SystemMessage(
            content="You are an AI assistant in an ongoing conversation. "
                   "Respond naturally based on the conversation history provided."
        )
        recent_messages = list(messages[-8:])  # Last 8 messages
        full_messages = [system_message] + recent_messages
    else:
        # Short conversations - use all messages
        full_messages = list(messages)
    
    print(f"🤖 Sending {len(full_messages)} messages to LLM")
    response = model.invoke(full_messages)
    
    return {"messages": [response]}

def create_smart_summary(state: State):
    """Create intelligent conversation summary preserving key context."""
    
    summary = state.get("summary", "")
    messages = list(state["messages"])
    
    print(f"📝 Creating summary from {len(messages)} messages")
    
    # Enhanced summarization prompt
    if summary:
        summary_prompt = (
            f"Current context summary: {summary}\n\n"
            "Please update this summary with the new conversation above. "
            "Focus on factual information, user details, projects, and key topics discussed. "
            "Keep it comprehensive but concise:"
        )
    else:
        summary_prompt = (
            "Please create a comprehensive summary of the conversation above. "
            "Include key information about the user, their interests, projects, and topics discussed. "
            "Focus on concrete details that would be useful for continuing the conversation:"
        )
    
    # Generate summary
    summarization_messages = messages + [HumanMessage(content=summary_prompt)]
    summary_response = model.invoke(summarization_messages)
    
    # Keep recent messages for context
    messages_to_keep = messages[-4:] if len(messages) > 4 else messages
    
    # Remove old messages
    messages_to_remove = []
    if len(messages) > 4:
        messages_to_remove = [RemoveMessage(id=m.id) for m in messages[:-4] if hasattr(m, 'id') and m.id is not None]
    
    print(f"✅ Summary created | Keeping {len(messages_to_keep)} recent messages")
    
    return {
        "summary": summary_response.content,
        "messages": messages_to_remove
    }

def should_summarize(state: State):
    """Determine if conversation should be summarized."""
    messages = state["messages"]
    
    if len(messages) > 8:
        print(f"📊 Conversation length: {len(messages)} messages → Summarizing")
        return "summarize_conversation"
    
    return END

print("✅ Enhanced memory logic functions defined")
print("🎯 Key features: Intelligent context framing, smart summarization, natural conversation flow")

✅ Enhanced memory logic functions defined
🎯 Key features: Intelligent context framing, smart summarization, natural conversation flow


## 🏗️ Graph Construction & Checkpointer Setup

In [168]:
def create_persistent_chatbot():
    """Create a chatbot with persistent memory using DynamoDBSaver."""
    
    # Initialize DynamoDB checkpointer
    checkpointer = DynamoDBSaver(
        table_name=TABLE_NAME,
        session=boto_session,
        ttl_seconds=TTL_SECONDS,
        s3_offload_config={
            "bucket_name": S3_BUCKET_NAME
        }
    )

    # Build conversation workflow
    workflow = StateGraph(State)
    
    # Add nodes
    workflow.add_node("conversation", call_model_with_memory)
    workflow.add_node("summarize_conversation", create_smart_summary)

    # Define flow
    workflow.add_edge(START, "conversation")
    workflow.add_conditional_edges("conversation", should_summarize)
    workflow.add_edge("summarize_conversation", END)

    # Compile with checkpointer for persistence
    graph = workflow.compile(checkpointer=checkpointer)
    
    return graph, checkpointer

# Create the persistent chatbot
persistent_chatbot, memory_checkpointer = create_persistent_chatbot()

print("✅ Persistent chatbot created with DynamoDBSaver")
print("🧠 Features: Auto-accumulating messages, intelligent summarization, cross-session memory")
print("🗄️ Storage: DynamoDB for metadata, S3 for large checkpoints (if configured)")

✅ Persistent chatbot created with DynamoDBSaver
🧠 Features: Auto-accumulating messages, intelligent summarization, cross-session memory
🗄️ Storage: DynamoDB for metadata, S3 for large checkpoints (if configured)


## 🚀 Chat Interface Function

In [169]:
def chat_with_persistent_memory(message: str, thread_id: str = "demo_user", graph_instance=None):
    """Chat with the bot using persistent memory across sessions."""
    
    if graph_instance is None:
        graph_instance = persistent_chatbot
    
    # Configuration for this conversation thread
    config = {"configurable": {"thread_id": thread_id}}
    
    # Create user message
    input_message = HumanMessage(content=message)
    
    # The magic happens here: DynamoDBSaver automatically:
    # 1. Retrieves existing conversation state from DynamoDB
    # 2. Merges with new message via add_messages annotation
    # 3. Processes through the enhanced memory logic
    # 4. Stores the updated state back to DynamoDB
    result = graph_instance.invoke({"messages": [input_message]}, config)
    
    # Get the assistant's response
    assistant_response = result["messages"][-1].content
    
    return assistant_response

print("✅ Chat interface ready with automatic state persistence")

✅ Chat interface ready with automatic state persistence


## 🎪 Interactive Demo

### Phase 1: Building Conversation Context

In [170]:
print("🎪 DEMO: Building Rich Conversation Context")
print("=" * 60)

# Use a demo thread for our conversation
demo_thread = "alice_ml_project"

# Step 1: User introduces themselves with detailed context
user_msg = "Hi! I'm Alice, a data scientist working on a neural network project about transformers and attention mechanisms for NLP."
response = chat_with_persistent_memory(user_msg, demo_thread)

print(f"👤 Alice: {user_msg}")
print(f"\n🤖 Assistant: {response}")
print("\n" + "="*60)

🎪 DEMO: Building Rich Conversation Context
🧠 Processing 1 messages | Summary: ❌
🤖 Sending 1 messages to LLM
👤 Alice: Hi! I'm Alice, a data scientist working on a neural network project about transformers and attention mechanisms for NLP.

🤖 Assistant: Hi Alice! It's great to meet you. Transformers and attention mechanisms are such a fascinating area of NLP - there's been incredible progress in recent years. 

What specific aspect of your transformer project are you working on? Are you:
- Building a model from scratch or fine-tuning an existing one?
- Focusing on a particular application like text classification, generation, or something else?
- Exploring modifications to the attention mechanism itself?
- Working on efficiency improvements or interpretability?

I'd be happy to discuss technical details, help troubleshoot issues, or brainstorm approaches depending on where you are in your project!



In [171]:
# Step 2: Adding more specific technical details
user_msg = "I'm particularly interested in how self-attention enables parallel processing compared to RNNs."
response = chat_with_persistent_memory(user_msg, demo_thread)

print(f"👤 Alice: {user_msg}")
print(f"\n🤖 Assistant: {response}")
print("\n" + "="*60)

🧠 Processing 3 messages | Summary: ❌
🤖 Sending 3 messages to LLM
👤 Alice: I'm particularly interested in how self-attention enables parallel processing compared to RNNs.

🤖 Assistant: Great question! The parallelization advantage of self-attention over RNNs is one of the key reasons transformers have been so transformative.

## RNN Sequential Bottleneck
In RNNs, you have this fundamental sequential dependency:
```
h₁ = f(x₁, h₀)
h₂ = f(x₂, h₁)  # Must wait for h₁
h₃ = f(x₃, h₂)  # Must wait for h₂
```
Each hidden state depends on the previous one, so you can't compute h₃ until h₂ is done, creating a sequential bottleneck that prevents parallelization across the sequence dimension.

## Self-Attention Parallelization
Self-attention computes all positions simultaneously:
```python
# All these operations are parallelizable
Q = XW_q  # All queries at once
K = XW_k  # All keys at once  
V = XW_v  # All values at once

# Attention scores for ALL positions computed in parallel
scores = QK^T / 

In [172]:
# Step 3: Discussing implementation challenges
user_msg = "I'm having trouble with the multi-head attention implementation. The computational complexity is concerning me."
response = chat_with_persistent_memory(user_msg, demo_thread)

print(f"👤 Alice: {user_msg}")
print(f"\n🤖 Assistant: {response}")
print("\n" + "="*60)

🧠 Processing 5 messages | Summary: ❌
🤖 Sending 5 messages to LLM
👤 Alice: I'm having trouble with the multi-head attention implementation. The computational complexity is concerning me.

🤖 Assistant: Multi-head attention complexity can definitely be tricky to manage! Let's break down where the computational costs come from and some strategies to address them.

## Complexity Breakdown
For multi-head attention with h heads, sequence length n, and model dimension d:

**Memory**: O(h × n²) for attention matrices - this is often the real bottleneck
**Compute**: O(h × n²d) total, but the n² term dominates for long sequences

## Common Implementation Issues

**1. Naive Head Processing**
```python
# Inefficient - separate operations per head
outputs = []
for i in range(num_heads):
    q_i = linear_q[i](x)  # d_model -> d_k
    k_i = linear_k[i](x)
    v_i = linear_v[i](x)
    attn_i = scaled_dot_product_attention(q_i, k_i, v_i)
    outputs.append(attn_i)
```

**2. Better: Batched Head Processi

### Phase 2: Triggering Summarization

In [173]:
print("📝 DEMO: Triggering Intelligent Summarization")
print("=" * 60)

# Add more messages to trigger summarization
conversation_topics = [
    "Can you explain the positional encoding used in transformers?",
    "How does the feed-forward network component work in each layer?",
    "What are the key differences between encoder and decoder architectures?",
    "I'm also working with BERT for downstream tasks. Any optimization tips?",
    "My current model has 12 layers. Should I consider more for better performance?"
]

for i, topic in enumerate(conversation_topics, 4):
    response = chat_with_persistent_memory(topic, demo_thread)
    print(f"\n💬 Message {i}: {topic}")
    print(f"🤖 Response: {response[:150]}...")
    
    # Show when summarization happens
    if i >= 6:
        print("📊 → Conversation length trigger reached - summarization may occur")

print("\n✅ Rich conversation context built with automatic summarization")

📝 DEMO: Triggering Intelligent Summarization
🧠 Processing 7 messages | Summary: ❌
🤖 Sending 8 messages to LLM

💬 Message 4: Can you explain the positional encoding used in transformers?
🤖 Response: Absolutely! Positional encoding is crucial because self-attention is inherently permutation-invariant - without it, the model can't distinguish betwee...
🧠 Processing 9 messages | Summary: ❌
🤖 Sending 9 messages to LLM
📊 Conversation length: 10 messages → Summarizing
📝 Creating summary from 10 messages
✅ Summary created | Keeping 4 recent messages

💬 Message 5: How does the feed-forward network component work in each layer?
🤖 Response: Great question! The feed-forward network (FFN) is a crucial but often overlooked component of each transformer layer. It's actually where most of the ...
🧠 Processing 5 messages | Summary: ✅
🤖 Sending 5 messages to LLM

💬 Message 6: What are the key differences between encoder and decoder architectures?
🤖 Response: Excellent question! This gets to the heart of

### Phase 3: Application Restart Simulation

In [174]:
print("🔄 DEMO: Simulating Application Restart")
print("=" * 60)
print("Creating completely new graph instance to simulate app restart...\n")

# Create a completely new graph instance (simulating app restart)
new_chatbot_instance, _ = create_persistent_chatbot()

print("✅ New chatbot instance created")
print("🧠 Memory should persist across instances via DynamoDBSaver\n")

🔄 DEMO: Simulating Application Restart
Creating completely new graph instance to simulate app restart...

✅ New chatbot instance created
🧠 Memory should persist across instances via DynamoDBSaver



### Phase 4: Memory Persistence Test

In [175]:
print("🧪 DEMO: Testing Memory Persistence After Restart")
print("=" * 60)

# Test memory with the new instance - this is the critical test
memory_test_msg = "Can you remind me about my transformer project and the specific challenges I mentioned?"
response = chat_with_persistent_memory(memory_test_msg, demo_thread, new_chatbot_instance)

print(f"👤 Alice: {memory_test_msg}")
print(f"\n🤖 Assistant: {response}")

# Analyze the response for memory indicators
memory_indicators = [
    "alice", "data scientist", "neural network", "transformer", 
    "attention mechanism", "nlp", "self-attention", "parallel processing",
    "multi-head attention", "computational complexity", "bert"
]

found_indicators = [indicator for indicator in memory_indicators if indicator in response.lower()]

print("\n" + "="*60)
print("🔍 MEMORY ANALYSIS:")
print(f"📊 Found {len(found_indicators)} memory indicators: {found_indicators[:5]}")

if len(found_indicators) >= 3:
    print("🎉 SUCCESS: Persistent memory is working perfectly!")
    print("✅ The assistant remembered detailed context across application restart")
else:
    print("⚠️  Memory persistence may need adjustment")
    print(f"Full response for analysis: {response}")

🧪 DEMO: Testing Memory Persistence After Restart
🧠 Processing 5 messages | Summary: ✅
🤖 Sending 5 messages to LLM
👤 Alice: Can you remind me about my transformer project and the specific challenges I mentioned?

🤖 Assistant: Based on our conversation, you're working on a neural network project with a dual focus:

## Your Current Setup
- **12-layer transformer model** that you're implementing (likely custom implementation rather than just fine-tuning)
- **BERT fine-tuning** for downstream NLP tasks running in parallel
- Working with **variable sequence lengths**

## Key Technical Challenges You've Mentioned

### **1. Multi-Head Attention Implementation Issues**
Your main bottleneck right now - you're having trouble with efficient multi-head attention implementation, specifically:
- Computational complexity concerns (the O(h × n²) memory for attention matrices)
- Need for batched vs naive head processing
- Memory bottlenecks that are impacting performance

### **2. Computational Complexi

### Phase 5: Advanced Memory Features

In [176]:
print("🚀 DEMO: Advanced Memory Features")
print("=" * 60)

# Test contextual follow-up questions
follow_up_msg = "Based on what we discussed, what would you recommend for optimizing my 12-layer BERT model?"
response = chat_with_persistent_memory(follow_up_msg, demo_thread, new_chatbot_instance)

print(f"👤 Alice: {follow_up_msg}")
print(f"\n🤖 Assistant: {response}")

print("\n" + "="*60)
print("💡 Advanced Features Demonstrated:")
print("✅ Contextual understanding across sessions")
print("✅ Natural conversation continuity")
print("✅ No 'I don't remember' responses")
print("✅ Intelligent context framing")
print("✅ Automatic state persistence in DynamoDB")

🚀 DEMO: Advanced Memory Features
🧠 Processing 7 messages | Summary: ✅
🤖 Sending 5 messages to LLM
👤 Alice: Based on what we discussed, what would you recommend for optimizing my 12-layer BERT model?

🤖 Assistant: Based on our discussions, here are my targeted recommendations for optimizing your 12-layer BERT model:

## **Priority 1: Multi-Head Attention Optimization**
Since this was your main bottleneck:

```python
# 1. Batched Multi-Head Processing
class OptimizedMultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_k = d_model // n_heads
        
        # Single linear layer for all heads (more efficient)
        self.qkv_proj = nn.Linear(d_model, 3 * d_model)
        self.output_proj = nn.Linear(d_model, d_model)
    
    def forward(self, x):
        batch_size, seq_len = x.shape[:2]
        
        # Compute Q, K, V for all heads at once
        qkv = self.q

## 🔍 Memory State Inspection

In [178]:
def inspect_conversation_state(thread_id: str = "demo_user"):
    """Inspect the current conversation state stored in DynamoDB."""
    
    config = {"configurable": {"thread_id": thread_id}}
    
    print(f"🔍 INSPECTING CONVERSATION STATE: {thread_id}")
    print("=" * 60)
    
    try:
        # Get state from current chatbot
        state = persistent_chatbot.get_state(config)
        
        if state and state.values:
            messages = state.values.get("messages", [])
            summary = state.values.get("summary", "")
            
            print(f"📊 CONVERSATION METRICS:")
            print(f"   • Total messages: {len(messages)}")
            print(f"   • Has summary: {'✅' if summary else '❌'}")
            print(f"   • Thread ID: {thread_id}")
            print(f"   • Storage: DynamoDB table '{TABLE_NAME}'")
            
            if summary:
                print(f"\n📝 CONVERSATION SUMMARY:")
                print(f"   {summary[:200]}...")
            
            print(f"\n💬 RECENT MESSAGES:")
            for i, msg in enumerate(messages[-3:]):
                msg_type = "👤" if isinstance(msg, HumanMessage) else "🤖"
                print(f"   {msg_type} {msg.content[:100]}...")
                
        else:
            print("❌ No conversation state found")
            
    except Exception as e:
        print(f"❌ Error inspecting state: {e}")

# Inspect our demo conversation
inspect_conversation_state(demo_thread)

🔍 INSPECTING CONVERSATION STATE: alice_ml_project
📊 CONVERSATION METRICS:
   • Total messages: 8
   • Has summary: ✅
   • Thread ID: alice_ml_project
   • Storage: DynamoDB table 'langgraph-checkpoints-ddb-717c7213'

📝 CONVERSATION SUMMARY:
   # Conversation Summary

## User Profile
- **Name**: Alice
- **Role**: Data scientist
- **Current Project**: Neural network project focused on transformers and attention mechanisms for NLP
- **Technica...

💬 RECENT MESSAGES:
   🤖 Based on our conversation, you're working on a neural network project with a dual focus:

## Your Cu...
   👤 Based on what we discussed, what would you recommend for optimizing my 12-layer BERT model?...
   🤖 Based on our discussions, here are my targeted recommendations for optimizing your 12-layer BERT mod...


## 🗑️ Cleanup: Delete Thread Data

In [179]:
def cleanup_thread(thread_id: str):
    """Delete all conversation data for a specific thread."""
    
    print(f"🗑️ CLEANING UP THREAD: {thread_id}")
    print("=" * 60)
    
    try:
        # Delete thread data from DynamoDB (and S3 if applicable)
        memory_checkpointer.delete_thread(thread_id)
        print(f"✅ Successfully deleted all data for thread: {thread_id}")
        print(f"   • Removed from DynamoDB table: {TABLE_NAME}")
        if S3_BUCKET_NAME:
            print(f"   • Removed from S3 bucket: {S3_BUCKET_NAME}")
    except Exception as e:
        print(f"❌ Error deleting thread: {e}")

# Uncomment to cleanup the demo thread
cleanup_thread(demo_thread)

🗑️ CLEANING UP THREAD: alice_ml_project
✅ Successfully deleted all data for thread: alice_ml_project
   • Removed from DynamoDB table: langgraph-checkpoints-ddb-717c7213
   • Removed from S3 bucket: langgraph-checkpoints-s3-717d768c


## 🎯 Demo Summary & Key Insights

In [180]:
print("🎯 PERSISTENT MEMORY CHATBOT - DEMO COMPLETE")
print("=" * 70)
print()
print("✨ WHAT WE ACCOMPLISHED:")
print("   🧠 Built rich conversation context with detailed user information")
print("   📝 Demonstrated automatic intelligent summarization")
print("   🔄 Simulated application restart with new graph instance")
print("   🎉 Proved persistent memory works across sessions")
print("   🚀 Showed natural conversation continuity without memory denial")
print()
print("🔧 KEY TECHNICAL COMPONENTS:")
print("   • DynamoDBSaver for reliable state persistence")
print("   • Enhanced context framing to avoid Claude's memory denial training")
print("   • Intelligent summarization preserving key conversation details")
print("   • Automatic message accumulation via add_messages annotation")
print("   • Cross-instance memory access through shared DynamoDB storage")
print()
print("🚀 PRODUCTION BENEFITS:")
print("   ⚡ Sub-second response times with DynamoDB")
print("   🔒 Reliable persistence with configurable TTL")
print("   📈 Scalable to millions of concurrent conversations")
print("   🛡️ Graceful handling of long conversation histories")
print("   🎯 Natural conversation flow without AI limitations")
print()
print("💡 NEXT STEPS:")
print("   • Customize summarization prompts for your domain")
print("   • Adjust conversation length thresholds")
print("   • Add conversation branching and context switching")
print("   • Implement user-specific memory isolation")
print("   • Add memory analytics and conversation insights")
print()
print("🎉 Ready for production deployment!")

🎯 PERSISTENT MEMORY CHATBOT - DEMO COMPLETE

✨ WHAT WE ACCOMPLISHED:
   🧠 Built rich conversation context with detailed user information
   📝 Demonstrated automatic intelligent summarization
   🔄 Simulated application restart with new graph instance
   🎉 Proved persistent memory works across sessions
   🚀 Showed natural conversation continuity without memory denial

🔧 KEY TECHNICAL COMPONENTS:
   • DynamoDBSaver for reliable state persistence
   • Enhanced context framing to avoid Claude's memory denial training
   • Intelligent summarization preserving key conversation details
   • Automatic message accumulation via add_messages annotation
   • Cross-instance memory access through shared DynamoDB storage

🚀 PRODUCTION BENEFITS:
   ⚡ Sub-second response times with DynamoDB
   🔒 Reliable persistence with configurable TTL
   📈 Scalable to millions of concurrent conversations
   🛡️ Graceful handling of long conversation histories
   🎯 Natural conversation flow without AI limitations

💡 NE